# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

1. [Откройте и изучите файл](#shag_1)
2. [Разбейте данные на выборки](#shag_2)
3. [Исследуйте модели. Модель дерево.](#shag_3)
4. [Модель случайный лес](#shag_4)
5. [Проверьте модели на тестовой выборке](#shag_5)
6. [Проверьте модели на адекватность](#shag_6)
7. [Выводы](#shag_7)

<a id='shag_1'></a>

## 1. Откройте и изучите файл

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="ticks", color_codes=True)
import datetime
import plotly.graph_objs as go
from scipy import stats as st
from scipy.stats import bartlett
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier


In [18]:
df = pd.read_csv('/datasets/users_behavior.csv')
print(df)

      calls  minutes  messages   mb_used  is_ultra
0      40.0   311.90      83.0  19915.42         0
1      85.0   516.75      56.0  22696.96         0
2      77.0   467.66      86.0  21060.45         0
3     106.0   745.53      81.0   8437.39         1
4      66.0   418.74       1.0  14502.75         0
...     ...      ...       ...       ...       ...
3209  122.0   910.98      20.0  35124.90         1
3210   25.0   190.36       0.0   3275.61         0
3211   97.0   634.44      70.0  13974.06         0
3212   64.0   462.32      90.0  31239.78         0
3213   80.0   566.09       6.0  29480.52         1

[3214 rows x 5 columns]


У моделей будет 4 признака: количество сообщений, мегабайт, минут и звонков, и 1 целевой параметр: тариф.

<a id='shag_2'></a>

## 2. Разбейте данные на выборки

In [19]:
features = df.drop('is_ultra',1)
target = df['is_ultra']

features2, features_test, target2, target_test = train_test_split(
    features, target, test_size=0.20, random_state=12345) 

features_train, features_valid, target_train, target_valid = train_test_split(
    features2, target2, test_size=0.25, random_state=12345) 

Разбил данные в пропорции 3:1:1 на обучающую выборку, валидационную и тестовую.

<a id='shag_3'></a>

## 3. Исследуйте модели

Исследуем модель дерево.

In [20]:
best_accuracy = 0
best_result = 10000
best_depth = 0
for depth in range(1, 30):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train,target_train)
    predictions_valid = model.predict(features_valid) 
    accuracy = accuracy_score(target_valid, predictions_valid)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        best_depth = depth
print('точность', best_accuracy)    
print('модель', best_model)
print('глубина', best_depth)

точность 0.7744945567651633
модель DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')
глубина 7


<a id='shag_4'></a>

Исследуем модель "случайный лес".

In [21]:
best_accuracy = 0
best_result = 10000
best_est = 0
best_depth = 0
for est in range(10, 121, 10):
    for depth in range (10, 50, 2):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train,target_train)
        predictions_valid = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predictions_valid)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model
            best_est = est
            best_depth = depth

print("Точность наилучшей модели на валидационной выборке:", best_accuracy, "Количество деревьев:", best_est,
      "Максимальная глубина:", depth)

Точность наилучшей модели на валидационной выборке: 0.80248833592535 Количество деревьев: 110 Максимальная глубина: 48


<a id='shag_5'></a>

## 4. Проверьте модель на тестовой выборке

In [22]:
model = RandomForestClassifier(random_state=12345, n_estimators=110, max_depth=48)
model.fit(features_train,target_train) 
predictions_test = model.predict(features_test)

accuracy = accuracy_score(target_test, predictions_test)
print("Точность наилучшей модели:", accuracy)

Точность наилучшей модели: 0.7884914463452566


<a id='shag_6'></a>

## 5. (бонус) Проверьте модели на адекватность

In [23]:
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=12345)
dummy_clf.fit(features_train,target_train) 
predictions_valid = dummy_clf.predict(features_test)
#dummy_clf.score(X, y)
accuracy = accuracy_score(target_test, predictions_valid)
print("Точность простой модели:", accuracy)

Точность простой модели: 0.6951788491446346


<a id='shag_7'></a>

## Выводы

Модель дерева показа точность равную 0.77, модель случайный лес - 0.80. В результате лучшей моделью признан случайный лес с параметрами Количество деревьев: 110 Максимальная глубина: 48. На тестовой выборке данная модель показала точность 0.788, что лучше простой модели с точностью 0.69.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
